# 2.5 KPIs Avanzados

**Resumen Ejecutivo**: Aprende a derivar métricas clave (ticket promedio, margen por segmento, crecimiento diario, retención) combinando agregaciones, window functions y CTEs.

---

## 🎯 Objetivos de Aprendizaje
- Calcular KPIs financieros (margen, ticket promedio) y de crecimiento (variación día a día).
- Aplicar `LAG` y `NULLIF` para manejar divisiones por cero y comparaciones temporales.
- Construir métricas de retención y participación por segmento.

## 🧰 Prerrequisitos
- Haber completado `04_modelado_dimensional_basico.ipynb`.
- Dominar window functions (`LAG`, `OVER`) y CTEs.
- Conocer agregaciones (`SUM`, `AVG`, `COUNT`) y `DECIMAL` para precisión.

---

## 📚 Conceptos Clave
- **Ticket promedio**: ingresos totales / número de transacciones.
- **Margen**: precio - costo; analizar por producto, segmento, región.
- **Crecimiento**: comparar período actual vs. anterior con `LAG` y `NULLIF`.
- **Retención**: clientes con compras recurrentes (al menos 2 días distintos).

In [ ]:
-- Margen promedio por producto
SELECT p.producto_id, AVG(p.precio_unitario - p.costo_unitario) AS margen_promedio
FROM dbo.dim_productos p
GROUP BY p.producto_id;

In [ ]:
-- Crecimiento de unidades día a día (window + expresión)
WITH diario AS (
  SELECT fecha, SUM(cantidad) AS unidades
  FROM dbo.fact_ventas
  GROUP BY fecha
)
SELECT fecha, unidades,
       unidades - LAG(unidades) OVER (ORDER BY fecha) AS delta_unidades,
       CASE WHEN LAG(unidades) OVER (ORDER BY fecha) = 0 THEN NULL
            ELSE CAST((unidades - LAG(unidades) OVER (ORDER BY fecha)) AS DECIMAL(10,2)) / NULLIF(LAG(unidades) OVER (ORDER BY fecha),0) END AS pct_crecimiento
FROM diario;

---

## 🧪 Ejercicios Guiados
- 🟢 Calcular ticket promedio (ingresos / transacciones).
- 🟠 Margen total y participación porcentual por segmento.
- 🔴 Calcular retención aproximada: clientes con ventas en al menos 2 días distintos / total clientes activos.

In [ ]:
SELECT CAST(SUM(fv.cantidad*p.precio_unitario*(1 - fv.descuento_pct/100.0)) AS DECIMAL(10,2)) / COUNT(*) AS ticket_promedio
FROM dbo.fact_ventas fv
JOIN dbo.dim_productos p ON fv.producto_id = p.producto_id;

In [ ]:
WITH margen_segmento AS (
  SELECT c.segmento, SUM(fv.cantidad*(p.precio_unitario-p.costo_unitario)) AS margen
  FROM dbo.fact_ventas fv
  JOIN dbo.dim_clientes c ON fv.cliente_id = c.cliente_id
  JOIN dbo.dim_productos p ON fv.producto_id = p.producto_id
  GROUP BY c.segmento
), total AS (SELECT SUM(margen) AS mt FROM margen_segmento)
SELECT segmento, margen, CAST(margen/mt*100 AS DECIMAL(5,2)) AS pct
FROM margen_segmento CROSS JOIN total;

---

## ⚠️ Errores Comunes
- Usar `COUNT(*)` cuando se requiere `COUNT(DISTINCT)` (sobrestima clientes activos).
- Dividir antes de agrupar (orden de operaciones incorrecto).
- No manejar divisiones por cero con `NULLIF` (errores en cálculos de crecimiento).
- Agrupar tras calcular KPIs complejos sin CTE intermedia (reduce legibilidad).

---

## ✅ Conclusiones
- KPIs avanzados requieren combinar agregaciones, window functions y manejo de nulos.
- `LAG` y `NULLIF` son esenciales para comparaciones temporales y evitar errores de división.
- CTEs modularizan cálculos complejos (ingresos → costos → margen) y mejoran auditoría.

---

## 🚀 Aplicación Práctica
- Dashboards ejecutivos con ticket promedio, margen por segmento, crecimiento diario.
- Alertas de métricas (ticket bajo umbral, margen negativo, crecimiento plano).
- Análisis de retención para campañas de fidelización.

---

## 💼 Perspectiva de Negocio
- Ticket promedio identifica oportunidades de upsell y cross-sell.
- Margen por segmento guía estrategia de pricing y portafolio.
- Crecimiento y retención son Early Warning Indicators (EWI) para ajuste de estrategia.

---

## 🔖 Pie Editorial

**Curso**: Fundamentos de SQL Server - Nivel 2  
**Módulo**: 2.5 KPIs Avanzados  
**Versión**: 2.0 (Actualizado Enero 2025)  
**Autor**: lraigosov / LuisRai  
**Licencia**: Uso educativo - Atribución requerida

> 💡 Nota sobre IA: Este material fue estructurado con asistencia de modelos de lenguaje (OpenAI GPT-4, Anthropic Claude); el contenido fue validado y curado por especialistas para evitar alucinaciones y asegurar aplicabilidad práctica.

---
## Navegación
[⬅️ Anterior](04_modelado_dimensional_basico.ipynb) | [Siguiente ➡️](06_segmentacion_basica.ipynb)
---
